# Regression: Canadian Oil and Gas Stock Price Prediction using Company Balance Sheets

### Introduction

In this project we want to create a stock price prediction model using company balance sheet values for various Canadian oil and gas companies. We will be using the Financial Modelling Prep (Financial Modelling Prep, 2024) and Python Yahoo Finance (Yfinance PyPI., 2023) API's to gather the data, then we would be utilizing Python for data wrangling and cleaning, then we would switch into R, where we would go ahead and attempt to create a robust regression model to predict the stock price off of a given oil and gas company's balance sheet values. From my accounting knowledge, it was decided that utilizing a balance sheet for stock price prediction would make the most sense as this is the one company financial statement which provides a snapshot in time, whereas other financial statements provide values over a certain period of time. This regression model would then be able to be used as a strategy of value investing within the Canadian oil and gas industry.

Value Investing: An investment strategy based on looking for stocks which are undervalued by the market at large (Caplinger D.,2023).

### Data Gathering

In [1]:
# importing relevant packages
import pandas as pd
import numpy as np
import requests
import yfinance as yf
# suppresses scientific notation for pandas dataframes (4 decimal places maximum)
pd.options.display.float_format = '{:.4f}'.format

In [2]:
# opening the api key text document and extracting the api key as a string variable
with open("api_key.txt") as api_doc:
    api_key = api_doc.read()

def get_balance_sheet(ticker):
    """
    returns the last 5 annual balance sheet values of a given stock ticker in a list of jsons
    """
    # extracting the balance sheet information from the fmp api in a json format
    balance_sheet = requests.get(f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?period=annualdatatype=csv&apikey={api_key}")
    balance_sheet = balance_sheet.json()

    return balance_sheet

In [3]:
# creating a list of relevant canadian oil and gas stock tickers
tickers = ["SU", "CVE", "CNQ", "TRP", "IMO", "ENB", "PPL", "KEY", "XOM", "CVX", "SHEL", "TTE", "COP", "BP", "MPC", "OXY", "PXD", "KMI", "WMB", "LNG", "MPLX", "SLB", "EOG", "SUN", "VLO", "OVV"]

# initializing an empty list to store the balance sheet values for all of the tickers and years
balance_sheets = []

# looping through each ticker in the list and extracting the balance sheet values and appending them to the balance_sheets list
for ticker in tickers:
    # extracting the current ticker's last 5 annual balance sheets in a list
    current_ticker_balance_sheets = get_balance_sheet(ticker)
    # looping through each balance sheet for the past 5 years for the current ticker and appending it to the balance_sheets list
    for balance_sheet in current_ticker_balance_sheets:
        balance_sheets.append(balance_sheet)

In [4]:
# converting the balance_sheets list to a pandas dataframe
balance_sheets_df = pd.DataFrame(balance_sheets)
balance_sheets_df.head()

,date,symbol,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,cashAndCashEquivalents,shortTermInvestments,...,totalStockholdersEquity,totalEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,link,finalLink
0,2022-12-31,SU,CAD,0000311337,2022-12-31,2022-12-30 19:00:00,2022,FY,1980000000,0,...,39367000000,39367000000,84618000000,0,84618000000,758000000,15619000000,13639000000,,
1,2021-12-31,SU,CAD,0000311337,2021-12-31,2021-12-31 00:00:00,2021,FY,2205000000,0,...,36614000000,36614000000,83739000000,0,83739000000,391000000,18354000000,16149000000,None,None
2,2020-12-31,SU,CAD,0000311337,2020-12-31,2020-12-31 00:00:00,2020,FY,1885000000,0,...,35757000000,35757000000,84616000000,0,84616000000,323000000,21699000000,19814000000,None,None
3,2019-12-31,SU,CAD,0000311337,2019-12-31,2019-12-31 00:00:00,2019,FY,1960000000,0,...,42042000000,42042000000,89435000000,0,89435000000,289000000,17970000000,16010000000,None,None
4,2018-12-31,SU,CAD,0000311337,2018-12-31,2018-12-30 19:00:00,2018,FY,2221000000,0,...,44005000000,44005000000,89579000000,0,89579000000,237000000,17350000000,15129000000,,


### Data Cleaning/Wrangling

In [5]:
# checking the number of missing values in each column
balance_sheets_df.isna().sum()

date                                        0
symbol                                      0
reportedCurrency                            0
cik                                         0
fillingDate                                 0
acceptedDate                                0
calendarYear                                0
period                                      0
cashAndCashEquivalents                      0
shortTermInvestments                        0
cashAndShortTermInvestments                 0
netReceivables                              0
inventory                                   0
otherCurrentAssets                          0
totalCurrentAssets                          0
propertyPlantEquipmentNet                   0
goodwill                                    0
intangibleAssets                            0
goodwillAndIntangibleAssets                 0
longTermInvestments                         0
taxAssets                                   0
otherNonCurrentAssets             

In [6]:
# dropping the columns with missing values as we do not need them for our analysis
balance_sheets_df.drop(columns = ["link", "finalLink"], inplace = True)

In [7]:
# checking the currency types in the dataframe
balance_sheets_df["reportedCurrency"].unique()

array(['CAD', 'USD'], dtype=object)

Looks like there are USD values in our dataset, however we want all numbers to be in CAD, therefore we must import the Bank of Canada historical Exchange Rates and convert the USD ones to CAD.

In [8]:
# importing the exchange rates csv essential columns only, and the exchange rate column as a floating point value (Bank of Canada, 2024)
exchange_rates_df = pd.read_csv("exchange_rates.csv", usecols = ["date", "FXUSDCAD"], dtype = {'FXUSDCAD': float})
# adding a usd column in order to join on it when joining with the balance_sheets_df dataframe
# to ensure only USD values have an exchange rate
exchange_rates_df["reportedCurrency"] = "USD"
exchange_rates_df.head()

,date,FXUSDCAD,reportedCurrency
0,2017-01-03,1.3435,USD
1,2017-01-04,1.3315,USD
2,2017-01-05,1.3244,USD
3,2017-01-06,1.3214,USD
4,2017-01-09,1.3240,USD


In [9]:
# left joining the balance_sheets_df and exchange_rates_df dataframes on the date and reportedCurrency columns
balance_sheets_df = pd.merge(balance_sheets_df, exchange_rates_df, on = ["date", "reportedCurrency"], how = "left")
# making the exchange rate 1 for stocks in CAD
balance_sheets_df.loc[balance_sheets_df["reportedCurrency"] == "CAD", "FXUSDCAD"] = 1

In [10]:
# checking for missing exchange rates
balance_sheets_df.loc[balance_sheets_df["FXUSDCAD"].isnull()]

,date,symbol,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,cashAndCashEquivalents,shortTermInvestments,...,othertotalStockholdersEquity,totalStockholdersEquity,totalEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,FXUSDCAD
30,2022-12-31,PPL,USD,0000922224,2023-02-17,2023-02-17 14:09:21,2021,FY,356000000,0,...,-967000000,13915000000,13918000000,37837000000,3000000,37837000000,0,14228000000,13872000000,NaN
35,2022-12-31,KEY,USD,0000091576,2023-02-22,2023-02-22 15:19:02,2022,FY,887000000,41549000000,...,376000000,13454000000,13454000000,189813000000,0,189813000000,56049000000,30079000000,29192000000,NaN
40,2022-12-31,XOM,USD,0000034088,2023-02-22,2023-02-22 16:37:14,2022,FY,29640000000,0,...,-240293000000,195049000000,202473000000,369067000000,7424000000,369067000000,49793000000,41193000000,11553000000,NaN
45,2022-12-31,CVX,USD,0000093410,2023-02-23,2023-02-23 11:51:02,2022,FY,17678000000,223000000,...,-29536000000,191856000000,192816000000,257709000000,960000000,257709000000,45461000000,23339000000,5661000000,NaN
50,2022-12-31,SHEL,USD,0001306965,2023-03-09,2023-03-09 06:06:12,2022,FY,40246000000,23864000000,...,20406000000,190472000000,192598000000,443024000000,2126000000,443024000000,27226000000,83795000000,43549000000,NaN
55,2022-12-31,TTE,USD,0000879764,2023-03-24,2023-03-24 12:13:13,2022,FY,33026000000,8746000000,...,-7554000000,111724000000,114570000000,303864000000,2846000000,303864000000,40417000000,60766000000,27740000000,NaN
60,2022-12-31,COP,USD,0001163165,2023-02-16,2023-02-16 12:42:43,2022,FY,6458000000,2785000000,...,953000000,48003000000,48003000000,93829000000,0,93829000000,11010000000,16643000000,10185000000,NaN
65,2022-12-31,BP,USD,0000313807,2023-03-10,2023-03-10 06:06:30,2022,FY,29195000000,578000000,...,28026000000,67553000000,82990000000,288120000000,15437000000,288120000000,23271000000,55493000000,26298000000,NaN
70,2022-12-31,MPC,USD,0001510295,2023-02-23,2023-02-23 13:32:52,2022,FY,8625000000,3145000000,...,1561000000,27715000000,28683000000,83500000000,968000000,83500000000,9611000000,27909000000,19284000000,NaN
75,2022-12-31,OXY,USD,0000797468,2023-02-27,2023-02-27 16:52:27,2022,FY,984000000,0,...,3409000000,30085000000,30085000000,72609000000,0,72609000000,3176000000,20765000000,19781000000,NaN


In [11]:
# appears that they are all from the same date of "2022-12-31," will get the exchange rate for "2022-12-30" and input it manually (Bank of Canada, 2022)
balance_sheets_df.loc[balance_sheets_df["FXUSDCAD"].isnull(), "FXUSDCAD"] = 1.3544

In [12]:
# creating a list of all column names that need to be converted to CAD
list_of_USD_cols = balance_sheets_df.columns.to_list()
list_of_USD_cols

['date',
 'symbol',
 'reportedCurrency',
 'cik',
 'fillingDate',
 'acceptedDate',
 'calendarYear',
 'period',
 'cashAndCashEquivalents',
 'shortTermInvestments',
 'cashAndShortTermInvestments',
 'netReceivables',
 'inventory',
 'otherCurrentAssets',
 'totalCurrentAssets',
 'propertyPlantEquipmentNet',
 'goodwill',
 'intangibleAssets',
 'goodwillAndIntangibleAssets',
 'longTermInvestments',
 'taxAssets',
 'otherNonCurrentAssets',
 'totalNonCurrentAssets',
 'otherAssets',
 'totalAssets',
 'accountPayables',
 'shortTermDebt',
 'taxPayables',
 'deferredRevenue',
 'otherCurrentLiabilities',
 'totalCurrentLiabilities',
 'longTermDebt',
 'deferredRevenueNonCurrent',
 'deferredTaxLiabilitiesNonCurrent',
 'otherNonCurrentLiabilities',
 'totalNonCurrentLiabilities',
 'otherLiabilities',
 'capitalLeaseObligations',
 'totalLiabilities',
 'preferredStock',
 'commonStock',
 'retainedEarnings',
 'accumulatedOtherComprehensiveIncomeLoss',
 'othertotalStockholdersEquity',
 'totalStockholdersEquity',
 '

In [13]:
# removing columns that do not have to be converted to CAD from the list of columns
elements_to_remove = ["date", "symbol", "reportedCurrency", "cik", "fillingDate", "acceptedDate", "calendarYear", "period", "FXUSDCAD"]
for element in elements_to_remove:
    list_of_USD_cols.remove(element)

In [14]:
# looping through the list of columns and converting the columns that are in USD to CAD
for column in list_of_USD_cols:
    balance_sheets_df[column] = balance_sheets_df[column]*balance_sheets_df["FXUSDCAD"]

In [15]:
# checking which columns are in the dataframe now
balance_sheets_df.columns

Index(['date', 'symbol', 'reportedCurrency', 'cik', 'fillingDate',
       'acceptedDate', 'calendarYear', 'period', 'cashAndCashEquivalents',
       'shortTermInvestments', 'cashAndShortTermInvestments', 'netReceivables',
       'inventory', 'otherCurrentAssets', 'totalCurrentAssets',
       'propertyPlantEquipmentNet', 'goodwill', 'intangibleAssets',
       'goodwillAndIntangibleAssets', 'longTermInvestments', 'taxAssets',
       'otherNonCurrentAssets', 'totalNonCurrentAssets', 'otherAssets',
       'totalAssets', 'accountPayables', 'shortTermDebt', 'taxPayables',
       'deferredRevenue', 'otherCurrentLiabilities', 'totalCurrentLiabilities',
       'longTermDebt', 'deferredRevenueNonCurrent',
       'deferredTaxLiabilitiesNonCurrent', 'otherNonCurrentLiabilities',
       'totalNonCurrentLiabilities', 'otherLiabilities',
       'capitalLeaseObligations', 'totalLiabilities', 'preferredStock',
       'commonStock', 'retainedEarnings',
       'accumulatedOtherComprehensiveIncomeLoss',
 

In [16]:
# removing columns which are not needed anymore
balance_sheets_df.drop(columns = ["reportedCurrency", "cik", "fillingDate", "acceptedDate", "calendarYear", "period"], inplace = True)

In [17]:
balance_sheets_df.head()

,date,symbol,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,propertyPlantEquipmentNet,...,othertotalStockholdersEquity,totalStockholdersEquity,totalEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,FXUSDCAD
0,2022-12-31,SU,1980000000.0000,0.0000,1980000000.0000,6312000000.0000,5058000000.0000,1186000000.0000,14536000000.0000,64649000000.0000,...,571000000.0000,39367000000.0000,39367000000.0000,84618000000.0000,0.0000,84618000000.0000,758000000.0000,15619000000.0000,13639000000.0000,1.0000
1,2021-12-31,SU,2205000000.0000,0.0000,2205000000.0000,4662000000.0000,4110000000.0000,0.0000,10977000000.0000,67772000000.0000,...,612000000.0000,36614000000.0000,36614000000.0000,83739000000.0000,0.0000,83739000000.0000,391000000.0000,18354000000.0000,16149000000.0000,1.0000
2,2020-12-31,SU,1885000000.0000,0.0000,1885000000.0000,3884000000.0000,3617000000.0000,0.0000,9386000000.0000,70416000000.0000,...,591000000.0000,35757000000.0000,35757000000.0000,84616000000.0000,0.0000,84616000000.0000,323000000.0000,21699000000.0000,19814000000.0000,1.0000
3,2019-12-31,SU,1960000000.0000,0.0000,1960000000.0000,4185000000.0000,3761000000.0000,0.0000,9906000000.0000,75068000000.0000,...,566000000.0000,42042000000.0000,42042000000.0000,89435000000.0000,0.0000,89435000000.0000,289000000.0000,17970000000.0000,16010000000.0000,1.0000
4,2018-12-31,SU,2221000000.0000,0.0000,2221000000.0000,3320000000.0000,3159000000.0000,0.0000,8700000000.0000,76564000000.0000,...,540000000.0000,44005000000.0000,44005000000.0000,89579000000.0000,0.0000,89579000000.0000,237000000.0000,17350000000.0000,15129000000.0000,1.0000


In [18]:
# checking the datatypes of each column in the dataframe
balance_sheets_df.dtypes

date                                        object
symbol                                      object
cashAndCashEquivalents                     float64
shortTermInvestments                       float64
cashAndShortTermInvestments                float64
netReceivables                             float64
inventory                                  float64
otherCurrentAssets                         float64
totalCurrentAssets                         float64
propertyPlantEquipmentNet                  float64
goodwill                                   float64
intangibleAssets                           float64
goodwillAndIntangibleAssets                float64
longTermInvestments                        float64
taxAssets                                  float64
otherNonCurrentAssets                      float64
totalNonCurrentAssets                      float64
otherAssets                                float64
totalAssets                                float64
accountPayables                

### Adding the Stock Price Column

In [19]:
# creating a list of relevant canadian oil and gas stock tickers
tickers = ["SU", "CVE", "CNQ", "TRP", "IMO", "ENB", "PPL", "KEY", "XOM", "CVX", "SHEL", "TTE", "COP", "BP", "MPC", "OXY", "PXD", "KMI", "WMB", "LNG", "MPLX", "SLB", "EOG", "SUN", "VLO", "OVV"]

# initializing an empty list to store the final merged dataframes (each element is for 1 stock ticker) to combine
merged_dfs = []

# looping through each stock ticker in the list above
for ticker in tickers:
    
    # initializing the variable for yahoo finance information for the current stock ticker
    temp_ticker_info = yf.Ticker(ticker)

    # extracting the stock price information from the yahoo finance api starting from the date "2015-01-01" as we only want recent information
    stock_price = temp_ticker_info.history(start = "2015-01-01", end = None)
    # removing the date index and renaming it to "date" as a regular column
    stock_price = stock_price.reset_index().rename(columns = {"Date":"date"})
    # removing the time from the date column
    stock_price["date"] = pd.to_datetime(stock_price["date"]).dt.date
    # converting the "date" column to a string datatype6
    stock_price["date"] = stock_price["date"].apply(lambda x: str(x))
    # filtering for only the "date" and "Close" (closing stock price) columns
    stock_price = stock_price[["date", "Close"]]
    # renaming the closing stock price column
    stock_price = stock_price.rename(columns = {"Close":"ClosingStockPrice"})
    # adding the ticker symbol as a column in the dataframe
    stock_price["symbol"] = ticker

    # appending the merged dataframe to the merged_dfs list
    merged_dfs.append(stock_price)
    
# concatenating all of the dataframes for each stock ticker stock price into one
stock_price_df = pd.concat(merged_dfs)

In [20]:
# displaying the stock price dataframe
stock_price_df.head()

,date,ClosingStockPrice,symbol
0,2015-01-02,23.0012,SU
1,2015-01-05,21.9791,SU
2,2015-01-06,21.6022,SU
3,2015-01-07,21.6384,SU
4,2015-01-08,21.9791,SU


In [21]:
# converting the datatype of date columns to datetime
balance_sheets_df["date"] = balance_sheets_df["date"].astype("datetime64[ns]")
stock_price_df["date"] = stock_price_df["date"].astype("datetime64[ns]")

In [22]:
def date_correction(date, stock_prices):
    """
    this function takes a date and then checks if it is present in another pandas series, if it is not,
    we subtract one day and then check again, over and over, until the date is present in the second
    series, and then returns this new date (important as balance sheets are not always recorded on stock
    trading days, therefore the best estimate would be the last stock value right before the balance sheet date)
    """
    while date not in stock_prices.values:
        date -= pd.DateOffset(days = 1)

    return date

# now we can go through and change the dates of balance sheets to ensure they are all on trading days
balance_sheets_df["date"] = balance_sheets_df["date"].apply(lambda x: date_correction(x, stock_price_df["date"]))

In [23]:
# joining the balance sheet dataframe and the stock price dataframe
balance_sheet_stock_prices_df = balance_sheets_df.merge(stock_price_df, how = "left", on = ["date", "symbol"])

In [24]:
balance_sheet_stock_prices_df.head()

,date,symbol,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,propertyPlantEquipmentNet,...,totalStockholdersEquity,totalEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,FXUSDCAD,ClosingStockPrice
0,2022-12-30,SU,1980000000.0000,0.0000,1980000000.0000,6312000000.0000,5058000000.0000,1186000000.0000,14536000000.0000,64649000000.0000,...,39367000000.0000,39367000000.0000,84618000000.0000,0.0000,84618000000.0000,758000000.0000,15619000000.0000,13639000000.0000,1.0000,30.2341
1,2021-12-31,SU,2205000000.0000,0.0000,2205000000.0000,4662000000.0000,4110000000.0000,0.0000,10977000000.0000,67772000000.0000,...,36614000000.0000,36614000000.0000,83739000000.0000,0.0000,83739000000.0000,391000000.0000,18354000000.0000,16149000000.0000,1.0000,22.8518
2,2020-12-31,SU,1885000000.0000,0.0000,1885000000.0000,3884000000.0000,3617000000.0000,0.0000,9386000000.0000,70416000000.0000,...,35757000000.0000,35757000000.0000,84616000000.0000,0.0000,84616000000.0000,323000000.0000,21699000000.0000,19814000000.0000,1.0000,14.7492
3,2019-12-31,SU,1960000000.0000,0.0000,1960000000.0000,4185000000.0000,3761000000.0000,0.0000,9906000000.0000,75068000000.0000,...,42042000000.0000,42042000000.0000,89435000000.0000,0.0000,89435000000.0000,289000000.0000,17970000000.0000,16010000000.0000,1.0000,27.6565
4,2018-12-31,SU,2221000000.0000,0.0000,2221000000.0000,3320000000.0000,3159000000.0000,0.0000,8700000000.0000,76564000000.0000,...,44005000000.0000,44005000000.0000,89579000000.0000,0.0000,89579000000.0000,237000000.0000,17350000000.0000,15129000000.0000,1.0000,22.6501


In [25]:
# now we need to convert the stock prices from USD to CAD using the FX rate
balance_sheet_stock_prices_df["ClosingStockPrice"] = balance_sheet_stock_prices_df["ClosingStockPrice"]*balance_sheet_stock_prices_df["FXUSDCAD"]
# dropping the symbol and FX rate columns as we do not need them anymore
balance_sheet_stock_prices_df.drop(columns = ["FXUSDCAD", "symbol"], inplace = True)
# displaying the final dataset
balance_sheet_stock_prices_df.head()

,date,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,propertyPlantEquipmentNet,goodwill,...,othertotalStockholdersEquity,totalStockholdersEquity,totalEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,ClosingStockPrice
0,2022-12-30,1980000000.0000,0.0000,1980000000.0000,6312000000.0000,5058000000.0000,1186000000.0000,14536000000.0000,64649000000.0000,2892000000.0000,...,571000000.0000,39367000000.0000,39367000000.0000,84618000000.0000,0.0000,84618000000.0000,758000000.0000,15619000000.0000,13639000000.0000,30.2341
1,2021-12-31,2205000000.0000,0.0000,2205000000.0000,4662000000.0000,4110000000.0000,0.0000,10977000000.0000,67772000000.0000,2892000000.0000,...,612000000.0000,36614000000.0000,36614000000.0000,83739000000.0000,0.0000,83739000000.0000,391000000.0000,18354000000.0000,16149000000.0000,22.8518
2,2020-12-31,1885000000.0000,0.0000,1885000000.0000,3884000000.0000,3617000000.0000,0.0000,9386000000.0000,70416000000.0000,2892000000.0000,...,591000000.0000,35757000000.0000,35757000000.0000,84616000000.0000,0.0000,84616000000.0000,323000000.0000,21699000000.0000,19814000000.0000,14.7492
3,2019-12-31,1960000000.0000,0.0000,1960000000.0000,4185000000.0000,3761000000.0000,0.0000,9906000000.0000,75068000000.0000,2892000000.0000,...,566000000.0000,42042000000.0000,42042000000.0000,89435000000.0000,0.0000,89435000000.0000,289000000.0000,17970000000.0000,16010000000.0000,27.6565
4,2018-12-31,2221000000.0000,0.0000,2221000000.0000,3320000000.0000,3159000000.0000,0.0000,8700000000.0000,76564000000.0000,2892000000.0000,...,540000000.0000,44005000000.0000,44005000000.0000,89579000000.0000,0.0000,89579000000.0000,237000000.0000,17350000000.0000,15129000000.0000,22.6501


### Dataset

date

cashAndCashEquivalents

shortTermInvestments

cashAndShortTermInvestments

netReceivables

inventory

otherCurrentAssets

totalCurrentAssets

propertyPlantEquipmentNet

goodwill

intangibleAssets

goodwillAndIntangibleAssets

longTermInvestments

taxAssets

otherNonCurrentAssets

totalNonCurrentAssets

otherAssets

totalAssets

accountPayables

shortTermDebt

taxPayables

deferredRevenue

otherCurrentLiabilities

totalCurrentLiabilities

longTermDebt

deferredRevenueNonCurrent

deferredTaxLiabilitiesNonCurrent

otherNonCurrentLiabilities

totalNonCurrentLiabilities

otherLiabilities

capitalLeaseObligations

totalLiabilities

preferredStock

commonStock

retainedEarnings

accumulatedOtherComprehensiveIncomeLoss

othertotalStockholdersEquity

totalStockholdersEquity

totalEquity

totalLiabilitiesAndStockholdersEquity

minorityInterest

totalLiabilitiesAndTotalEquity

totalInvestments

totalDebt

netDebt

ClosingStockPrice

### Exporting

In [26]:
# exporting the dataset to a csv
balance_sheet_stock_prices_df.to_csv("balance_sheet_stock_prices.csv")

Now, we can move into R and conduct our regression modelling.

### References

Bank of Canada (2024), Daily Exchange rates, https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates/

Bank of Canada (2022), Daily Exchange rates: Lookup tool, https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates-lookup/?lookupPage=lookup_daily_exchange_rates_2017.php&startRange=2017-01-01&series%5B%5D=FXUSDCAD&lookupPage=lookup_daily_exchange_rates_2017.php&startRange=2017-01-01&rangeType=range&rangeValue=&dFrom=2022-12-29&dTo=2023-01-01&submit_button=Submit

Caplinger, D. (2023). The basics of value investing strategy. The Motley Fool. https://www.fool.com/investing/stock-market/types-of-stocks/value-stocks/value-investing-guide/#:~:text=Value%20investing%20is%20an%20investment,and%20earnings%20from%20their%20businesses. 

Financial Modelling Prep (2024), Free stock market API and financial statements API, https://site.financialmodelingprep.com/developer/docs

Yfinance. PyPI. (2023). https://pypi.org/project/yfinance/